<a href="https://colab.research.google.com/github/AminRj66/BCI2021_Spring_Ecog_team29/blob/master/Sentiment_Binance_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-binance

In [ ]:
from binance.client import Client
import datetime
import time
import pandas as pd
import numpy as np
import json
import logging

config = json.load(open("config/binance_conf.json"))
api_key = config["APIKey"]
secret_key = config["secretKey"]
update_interval = config["updateInterval"]
interval_type = config["intervalType"]
contractType = config["contractType"]
pair = config["Pair"]
timeframe = config["timeFrame"]
limit = config["limit"]
max_retries = config["maxRetries"]
delay = config["retry_delay"]


def make_connection(api_key, secrect_key, logger, maxRetries, retry_delay):
  num_retries = 0
  while True:
    try:
      num_retries +=1
      print('Attempting to connect...')
      myclient = Client(api_key, secrect_key)
      print('Connection established.')
      return myclient
    except Exception as e:
      print('Connection failed.')
      logger.error('error in making connection.')
      logger.error(str(e))
      if num_retries > maxRetries:
        raise
      time.sleep(retry_delay)

def request_binance(client, pair, param1, param2, param3, logger, maxRetries, retry_delay):
  num_retries = 0
  while True:
    try:
      if pair[-4:] == 'USDT':
        oih = client._request_futures_data_api('get', 'openInterestHist', data=param1)
        oih_ext = np.array([np.array([datetime.datetime.fromtimestamp(item['timestamp']/1000).strftime('%Y-%m-%d %H:%M:%S'),\
                          item['timestamp'], item['sumOpenInterest'], item['sumOpenInterestValue']]) for item in oih])
      
        lsh = client._request_futures_data_api('get', '/globalLongShortAccountRatio', data=param1)
        lsh_ext = np.array([np.array([datetime.datetime.fromtimestamp(item['timestamp']/1000).strftime('%Y-%m-%d %H:%M:%S'),\
                          item['timestamp'], item['longShortRatio'], item['longAccount'], item['shortAccount']]) for item in lsh])
      
        shp = client._request_futures_data_api('get', '/topLongShortPositionRatio', data=param1)
        shp_ext = np.array([np.array([datetime.datetime.fromtimestamp(item['timestamp']/1000).strftime('%Y-%m-%d %H:%M:%S'),\
                          item['timestamp'], item['longShortRatio'], item['longAccount'], item['shortAccount']]) for item in shp])
      
        sha = client._request_futures_data_api('get', '/topLongShortAccountRatio', data=param1)
        sha_ext = np.array([np.array([datetime.datetime.fromtimestamp(item['timestamp']/1000).strftime('%Y-%m-%d %H:%M:%S'),\
                          item['timestamp'], item['longShortRatio'], item['longAccount'], item['shortAccount']]) for item in sha])
      else:
        oih = client.futures_coin_open_interest_hist(**param3)
        oih_ext = np.array([np.array([datetime.datetime.fromtimestamp(item['timestamp'] / 1000).strftime('%Y-%m-%d %H:%M:%S'), \
                          item['timestamp'], item['sumOpenInterest'], item['sumOpenInterestValue']]) for item in oih])

        lsh = client._request_futures_coin_data_api("get", "globalLongShortAccountRatio", data=param2)
        lsh_ext = np.array([np.array([datetime.datetime.fromtimestamp(item['timestamp'] / 1000).strftime('%Y-%m-%d %H:%M:%S'), \
                          item['timestamp'], item['longShortRatio'], item['longAccount'], item['shortAccount']]) for item in lsh])

        shp = client._request_futures_coin_data_api("get", "topLongShortPositionRatio", data=param2)
        shp_ext = np.array([np.array([datetime.datetime.fromtimestamp(item['timestamp'] / 1000).strftime('%Y-%m-%d %H:%M:%S'), \
                          item['timestamp'], item['longShortRatio'], item['longPosition'], item['shortPosition']]) for item in shp])

        sha = client._request_futures_coin_data_api("get", "topLongShortAccountRatio", data=param2)
        sha_ext = np.array([np.array([datetime.datetime.fromtimestamp(item['timestamp'] / 1000).strftime('%Y-%m-%d %H:%M:%S'), \
                          item['timestamp'], item['longShortRatio'], item['longAccount'], item['shortAccount']]) for item in sha])

      data = np.hstack([oih_ext, lsh_ext[:,2:], shp_ext[:,2:], sha_ext[:,2:]])
      df = pd.DataFrame(data, columns=['DateTime', 'TimeStamp', 'sumOpenInterest', 'sumOpenInterestValue', 'globalLongShortAcc', 'globalLongAcc',\
                          'globalShortAcc','topLongShortPos','topLongPos', 'topShortPos', 'topLongShortAcc', 'topLongAcc', 'topShortAcc'])
      return df
    except Exception as e:
      print('Reading failed.')
      logger.error('Error in binance reading sentiment.')
      logger.error(str(e))
      if num_retries > maxRetries:
        raise
      time.sleep(retry_delay)


def get_market_sentiment(myclient, updateInterval, intervalType, contract, pair, logger, maxRetries, timeFrame, limit, retry_delay):
  t_lastUpdate = datetime.datetime(2020,1,1,0,0,0,0)
  read = 0
  param1 = {'symbol': pair, 'period' : timeFrame, 'limit':limit}
  param2 = {'pair': pair, 'period': timeFrame, 'limit': limit}
  param3 = {'pair': pair, 'contractType': contract, 'period': timeFrame, 'limit': limit}
  
  if intervalType == 'min':
    delta = datetime.timedelta(minutes = updateInterval)
  else:
    delta = datetime.timedelta(hours = updateInterval)

  while True:
    t_now = datetime.datetime.now()
    if t_now - t_lastUpdate >= delta:
      df0 = request_binance(client = myclient, pair = pair, param1 = param1, param2 = param2, param3 = param3, logger = logger, maxRetries=maxRetries, retry_delay = retry_delay)
      read+=1
      if read == 1:
        mydf = df0
      else:
        mydf = pd.concat([mydf,df0]).drop_duplicates(subset ='TimeStamp', keep='first')
      
      mydf.to_csv(f'content/sentiment_{pair}.csv', index=False)
      print(f"{pair} sentiment reading no.{read} is done: Current update time is {t_now}")
      t_lastUpdate = t_now
    else:
      time.sleep(5*60)
  return mydf


logging.basicConfig(filename="log/log.log", format='[%(asctime)s][%(levelname)s]: %(message)s',\
                    filemode='a', datefmt='%Y-%m-%d %H:%M:%S', level=logging.INFO)
logger = logging.getLogger()

client = make_connection(api_key, secret_key, logger=logger, maxRetries=max_retries, retry_delay = delay)

df = get_market_sentiment(myclient=client, updateInterval=update_interval, intervalType=interval_type, contract=contractType,\
          pair=pair, logger = logger, maxRetries = max_retries, timeFrame=timeframe, limit=limit, retry_delay = delay)


In [ ]:
!git init
!git config — global user.email “amin.ranjbar1988@gmail.com”
!git config — global user.name “AminRj66”

In [22]:
!git remote add origin https://github.com/AIBOXIA/AIT-fetching_binance_sentiment.git
!git push -u origin main

fatal: remote origin already exists.
error: src refspec main does not match any.
error: failed to push some refs to 'https://github.com/AIBOXIA/AIT-fetching_binance_sentiment.git'
